In [8]:
import warnings
warnings.filterwarnings(action="ignore")
import numpy as np
import pandas as pd
from tqdm import tqdm
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [9]:
client = chromadb.PersistentClient(
    path="db-llm-02",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

In [10]:
try:
    collection = client.get_collection(name="embeddings_collection_train")
except:
    collection = False

In [11]:
windows_size_opt = 112
y_train_df = pd.read_csv(f'../data/train_value_min_label_windows_{windows_size_opt}_llm.csv')
y_test_df = pd.read_csv(f'../data/test_value_min_label_windows_{windows_size_opt}_llm.csv')

In [12]:
# Path to the file with embeddings
progress_file = '../data/v6/progress2.txt'

results_list = list()
# Process the file line by line to minimize memory usage
with open(progress_file, 'r') as f:
    for i, line in tqdm(enumerate(f)):
        # Convert the line to an embedding
        embedding = np.fromstring(line.strip(), sep=',', dtype=np.float32)
        doc_id = f'doc_{i}'
        results = collection.query(query_embeddings=[embedding.tolist()], n_results=5)
        results['query_id'] = doc_id
        results_list.append(results)
        # Clean up any temporary variables or cache
        del results

        # Force garbage collection to free up memory (especially important in long loops)
        import gc
        gc.collect()

0it [00:00, ?it/s]

10682it [38:17,  4.65it/s]


In [13]:
results_df = pd.DataFrame(results_list)

In [14]:
results_df['ids'].value_counts()

ids
[[doc_23850, doc_23827, doc_23828, doc_23829, doc_23830]]    5562
[[doc_23822, doc_23827, doc_23823, doc_23824, doc_23825]]    5106
[[doc_12225, doc_12235, doc_12236, doc_12202, doc_12238]]      11
[[doc_1480, doc_2081, doc_2187, doc_1781, doc_23822]]           1
[[doc_1480, doc_1781, doc_1577, doc_23822, doc_23827]]          1
[[doc_2187, doc_2202, doc_2180, doc_1480, doc_1781]]            1
Name: count, dtype: int64